In [1]:
import pandas as pd
import pickle
import sys
import glob
import torch

In [13]:
sys.path.insert(0, "../../")
from prediction_models.att_mil.utils import checkpoint_utils
from prediction_models.att_mil.mil_models import mil
from prediction_models.att_mil.test_model import TestParams
from prediction_models.att_mil.utils import reinhard_fast as reinhard_bg

In [3]:
def load_opts(ckp_dir, data_dir, cuda, num_workers):
    model_opts = pickle.load(open(f"{ckp_dir}/options.pkl", "rb"))
    model_opts.cuda = cuda
    model_opts.num_workers = num_workers
    model_opts.data_dir = data_dir
    model_opts.device = "gpu" if cuda else "cpu"
    return model_opts

In [4]:
def load_model(ckp_dir, device, load_best):
    if load_best:
        ckp_path = f"{ckp_dir}/checkpoint_best.pth"
    else:
        ckp_path = f"{ckp_dir}/checkpoint.pth"

    if device == 'cuda':
        ckp = torch.load(ckp_path)
    else:
        ckp = torch.load(ckp_path, map_location=lambda storage, loc: storage)
    hp = ckp['hyperparams']
    params = ckp['model']
    base_encoder, feature_dim = \
            mil.config_encoder(hp['input_size'], hp['mil_params']['n_tile_classes'],
                               hp['encoder_arch'], False)
    model = mil.AttMILBatch(base_encoder, hp['pretrained'], hp['encoder_arch'], hp['input_size'],
                                        feature_dim, hp['mil_params'])
    model.load_state_dict(params)
    return model

def load_models(exp_dir, device, load_best=True):
    exp_fold_dirs = glob.glob(f"{exp_dir}/*/")
    models = []
    for cur_fold_dir in exp_fold_dirs:
        cur_model = load_model(cur_fold_dir, cuda, load_best)
        models.append(cur_model)
    return models


## Configure options
    

In [4]:
meandstd = {'mean': [0.8992915, 0.79110736, 0.8844037],'std': [0.13978645, 0.2604748, 0.14999403]}

In [5]:
slides_dir = "/home/jiayun/Documents/deepstorage/PANDA_challenge/train_images/"
slides_df = pd.read_csv("/home/jiayun/Documents/deepstorage/PANDA_challenge/train.csv")
slides_df = slides_df.sample(4)

In [9]:
load_best, cuda, num_workers = True, False, 5
dw_rate, ts_thres, overlap, top_n = 16, 0.1, 0.125, 20

In [10]:
model_dir = "../../cache/dw_16/resNeXt_ssl_epoch30_mse/"
opts = load_opts(model_dir, slides_dir, cuda, num_workers)

In [12]:
test_params = TestParams(slides_dir, opts.im_size, opts.input_size, dw_rate, ts_thres, overlap, top_n)

## Load train models

In [32]:
predictors = load_models(model_dir, opts.device, load_best)

../../cache/dw_16/resNeXt_ssl_epoch30_mse/0/


[]

## Run test

In [ ]:
def test(models, meanstd, test_slides_df, test_params, num_workers, device):
    normalize = T.Compose([
        T.Resize(test_params.input_size, interpolation=Image.ANTIALIAS),
        T.ToTensor(),
        T.Normalize(mean=meanstd['mean'], std=meanstd['std'])])
    tile_normalizer = reinhard_bg.ReinhardNormalizer()
    # use the pre-computed LAB mean and std values
    tile_normalizer.fit(None)
    dataset = test_slides.BiopsySlides(test_params, test_slides_df, normalize, tile_normalizer)
    loader = \
        torch.utils.data.DataLoader(dataset=dataset, batch_size=1, shuffle=False, drop_last=False,
                                    num_workers=num_workers, pin_memory=False)
    print("Start apply model")
    pred_data = []
    test_iter = iter(loader)
    with torch.no_grad():
        for step in range(len(loader)):
            tiles, image_id = test_iter.next()
            tiles = torch.squeeze(tiles, dim=0)
            image_id = str(image_id[0])
            tiles = tiles.to(device)
            slide_probs, _, _ = model(tiles)
            _, predicted = torch.max(slide_probs.data, 1)
            predicted = int(predicted.item())
            pred_data.append({"image_id": image_id, "isup_grade": predicted})
            del tiles

    pred_df = pd.DataFrame(columns=["image_id", "isup_grade"], data=pred_data)
    return pred_df